# 一次元CNNによる学習と識別性能評価（TDA）

---
  
引数：raw_al45.csv/raw_al135.csvがあるディレクトリまでのパス  
  
---
  
入力：raw_al45.csv/raw_al135.csv

---
  
出力：ACCURACY[loo or k_cv]_TDAautocor_1dCNN.csv　識別性能評価結果一覧  

---  
  
1dCNNを用いて学習し，交差検証法（k-分割交差検証，leave-one-out交差検証）を用いて識別性能評価を行う．  
ベクトル：各ボクセルの時系列データ

In [1]:
print('########### ML_1dCNN_TDAautocor.py program excution ############')

########### ML_1dCNN_TDAautocor.py program excution ############


In [2]:
import pandas as pd
import sys

# matplotlib inline部分は.pyの時にはコメントアウトしないとエラー出る！

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv1D, UpSampling1D
from keras.layers.pooling import MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


コマンドライン引数でraw_al45.csv/raw_al135.csvファイルがあるディレクトリまでのパスを取得

In [3]:
#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../Active/20181119mt/RawData/'

# 検証手法
col_name = 'leave-one-out'

# 機械学習するデータ（csvファイルのパラメータ名前）
PM_kizami = ['100', '300']
PM_hole = ['01dim', '012dim']

## ONEdCNN_LOO関数

引数として教師データをX，ラベルをyで受け取る．  
交差検証法の一つleave-one-out交差検証で識別精度評価を行う．

* (1個をテストデータ，残りを教師データにして学習・評価) * すべてのデータ個
* 得られたすべてのデータ個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をmain関数に返す

In [4]:
def ONEdCNN_LOO(X, y):

    # 識別率を格納する配列
    LOOscore = np.zeros(len(X))
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]

    
    # 1個をテストデータ，残りを教師データにして学習・評価
    # すべてのデータに対して行う
    for i in range(len(X)):
        
        print('------ ' + str(i) + ' / ' + str(len(X)) + '回 -----')

        # テストデータ
        X_test = X[i]
        y_test = y[i]

        # テストデータとして使用するデータを除いた教師データを作成
        X_train = np.delete(X, i, 0)
        y_train = np.delete(y, i, 0)


        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))
        
        # ダミー変数に変換：何分類するかによって数字を書き換える
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test, 2)


        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        
        # 何分類するかによってunitsを書き換える
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        LOOscore[i] = score[1]
        

        
    # 評価結果（識別率）の平均を求める
    result = LOOscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print(str(LOOscore) + '\n')

    return result

## ONEdCNN_kCV関数

引数として教師データをX，ラベルをyで受け取る．  
交差検証法の一つk-分割交差検証で識別精度評価を行う．

* 学習
* (k分割し，1グループをテストデータ，残りグループを教師データにして評価) * k
* 得られたk個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をmain関数に返す

In [5]:
def ONEdCNN_kCV(X, y):
    
    
    # 識別率を格納する配列
    kCVscore = np.zeros(cv_k)
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]
    
    # 分割数
    kf = KFold(n_splits=cv_k, shuffle=False)
    # 繰り返し回数
    i = 0

    for train_index, eval_index in kf.split(X):
        
        print('------ ' + str(i) + ' / ' + str(cv_k) + '回 -----')
        
        X_train, X_test = X[train_index], X[eval_index]
        y_train, y_test = y[train_index], y[eval_index]

        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))

        # ダミー変数に変換：何分類するかによって数字を書き換える
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test, 2)


        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        
        # 何分類するかによってunitsを書き換える
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        kCVscore[i] = score[1]
        
        i = i + 1
        

    # 評価結果（識別率）の平均を求める
    result = kCVscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print('k = ' + str(cv_k) + '：' + str(kCVscore))

    return result

## TrainingData関数
引数として読み込みたいタスクごとのデータをdata1/data2で受け取る． 
* 機械学習にかけれるようにデータのベクトル化とラベルを作成
* ベクトル化したデータとラベルをmain関数に返す

In [6]:
def TrainingData(data1, data2):

    # 各タスクのデータを縦結合
    all_data = pd.concat([data1, data2], axis = 0)

    # ベクトル化
    X = all_data.as_matrix()

    # ラベル作成 data1 = 0, data2 = 1
    label_data1 = np.zeros(len(data1.index))
    label_data2 = np.ones(len(data2.index))

    y = np.r_[label_data1, label_data2]

    
    return X, y

## main関数

In [7]:
if __name__ == '__main__':
    
        
    for kizami in range(len(PM_kizami)):
        
        for hole in range(len(PM_hole)):
            
            PM_dataName = PM_hole[hole] + PM_kizami[kizami]

            # 機械学習するデータ（提案手法でベクトル化したcsvファイル）のパス
            PATH_al45 = PATH + 'TDAvec_autocor_al45_' + PM_dataName + '.csv'
            PATH_al135 = PATH + 'TDAvec_autocor_al135_' + PM_dataName + '.csv'
            
            # csvファイルの読み込み
            al45 = pd.read_csv(PATH_al45, header = 0, index_col = 0)
            al135 = pd.read_csv(PATH_al135, header = 0, index_col = 0)
            
            
            # データとラベルの作成
            data, labels = TrainingData(al45, al135)
            
            # 学習とleave-one-out交差検証法
    
            print('leave-one-out Cross-Validation : ' + PM_dataName) 

            result_loo = ONEdCNN_LOO(data, labels)

            # データフレーム化
            result_loo = pd.DataFrame({col_name:[result_loo]}, index = ['TDA(' + PM_dataName + ') + 1dCNN'])
            print(result_loo)
            
            # csv書き出し
            PATH_RESULT = PATH + 'ACCURACY[loo]_TDAautocor' + PM_dataName + '_1dCNN.csv'
            result_loo.to_csv(PATH_RESULT, index = True)

leave-one-out Cross-Validation : 01dim100
------ 0 / 56回 -----
Epoch 1/100
55/55 [==============================] - 0s 2ms/step - loss: 0.5091 - acc: 0.4909
Epoch 2/100
55/55 [==============================] - 0s 244us/step - loss: 0.5091 - acc: 0.4909
Epoch 3/100
55/55 [==============================] - 0s 235us/step - loss: 0.5091 - acc: 0.4909
Epoch 4/100
55/55 [==============================] - 0s 255us/step - loss: 0.5091 - acc: 0.4909
Epoch 5/100
55/55 [==============================] - 0s 256us/step - loss: 0.5091 - acc: 0.4909
Epoch 6/100
55/55 [==============================] - 0s 224us/step - loss: 0.5091 - acc: 0.4909
Epoch 7/100
55/55 [==============================] - 0s 251us/step - loss: 0.5091 - acc: 0.4909
Epoch 8/100
55/55 [==============================] - 0s 260us/step - loss: 0.5091 - acc: 0.4909
Epoch 9/100
55/55 [==============================] - 0s 258us/step - loss: 0.5091 - acc: 0.4909
Epoch 10/100
55/55 [==============================] - 0s 258us/step - loss:

55/55 [==============================] - 0s 244us/step - loss: 0.5091 - acc: 0.4909
Epoch 85/100
55/55 [==============================] - 0s 250us/step - loss: 0.5091 - acc: 0.4909
Epoch 86/100
55/55 [==============================] - 0s 261us/step - loss: 0.5091 - acc: 0.4909
Epoch 87/100
55/55 [==============================] - 0s 235us/step - loss: 0.5091 - acc: 0.4909
Epoch 88/100
55/55 [==============================] - 0s 256us/step - loss: 0.5091 - acc: 0.4909
Epoch 89/100
55/55 [==============================] - ETA: 0s - loss: 0.4375 - acc: 0.562 - 0s 281us/step - loss: 0.5091 - acc: 0.4909
Epoch 90/100
55/55 [==============================] - ETA: 0s - loss: 0.5000 - acc: 0.500 - 0s 275us/step - loss: 0.5091 - acc: 0.4909
Epoch 91/100
55/55 [==============================] - 0s 286us/step - loss: 0.5091 - acc: 0.4909
Epoch 92/100
55/55 [==============================] - 0s 288us/step - loss: 0.5091 - acc: 0.4909
Epoch 93/100
55/55 [==============================] - 0s 248us/s

55/55 [==============================] - 0s 213us/step - loss: 0.2324 - acc: 0.6727
Epoch 67/100
55/55 [==============================] - 0s 247us/step - loss: 0.2324 - acc: 0.6364
Epoch 68/100
55/55 [==============================] - 0s 237us/step - loss: 0.2317 - acc: 0.6364
Epoch 69/100
55/55 [==============================] - 0s 234us/step - loss: 0.2312 - acc: 0.6545
Epoch 70/100
55/55 [==============================] - 0s 221us/step - loss: 0.2306 - acc: 0.6545
Epoch 71/100
55/55 [==============================] - 0s 269us/step - loss: 0.2299 - acc: 0.6909
Epoch 72/100
55/55 [==============================] - 0s 248us/step - loss: 0.2296 - acc: 0.6909
Epoch 73/100
55/55 [==============================] - 0s 242us/step - loss: 0.2294 - acc: 0.7091
Epoch 74/100
55/55 [==============================] - 0s 292us/step - loss: 0.2291 - acc: 0.6727
Epoch 75/100
55/55 [==============================] - 0s 236us/step - loss: 0.2286 - acc: 0.6545
Epoch 76/100
55/55 [=======================

55/55 [==============================] - 0s 266us/step - loss: 0.2054 - acc: 0.7455
Epoch 50/100
55/55 [==============================] - 0s 271us/step - loss: 0.2060 - acc: 0.6909
Epoch 51/100
55/55 [==============================] - 0s 247us/step - loss: 0.2057 - acc: 0.7091
Epoch 52/100
55/55 [==============================] - 0s 266us/step - loss: 0.2035 - acc: 0.6909
Epoch 53/100
55/55 [==============================] - 0s 273us/step - loss: 0.2031 - acc: 0.7273
Epoch 54/100
55/55 [==============================] - 0s 235us/step - loss: 0.2014 - acc: 0.7091
Epoch 55/100
55/55 [==============================] - 0s 244us/step - loss: 0.1992 - acc: 0.7273
Epoch 56/100
55/55 [==============================] - 0s 244us/step - loss: 0.1994 - acc: 0.7636
Epoch 57/100
55/55 [==============================] - 0s 239us/step - loss: 0.1979 - acc: 0.7455
Epoch 58/100
55/55 [==============================] - 0s 250us/step - loss: 0.1971 - acc: 0.7636
Epoch 59/100
55/55 [=======================

55/55 [==============================] - 0s 265us/step - loss: 0.4925 - acc: 0.5091
Epoch 33/100
55/55 [==============================] - 0s 276us/step - loss: 0.4924 - acc: 0.5091
Epoch 34/100
55/55 [==============================] - 0s 267us/step - loss: 0.4923 - acc: 0.5091
Epoch 35/100
55/55 [==============================] - 0s 291us/step - loss: 0.4923 - acc: 0.5091
Epoch 36/100
55/55 [==============================] - 0s 301us/step - loss: 0.4923 - acc: 0.5091
Epoch 37/100
55/55 [==============================] - 0s 317us/step - loss: 0.4922 - acc: 0.5091
Epoch 38/100
55/55 [==============================] - 0s 268us/step - loss: 0.4922 - acc: 0.5091
Epoch 39/100
55/55 [==============================] - 0s 290us/step - loss: 0.4921 - acc: 0.5091
Epoch 40/100
55/55 [==============================] - 0s 248us/step - loss: 0.4921 - acc: 0.5091
Epoch 41/100
55/55 [==============================] - 0s 298us/step - loss: 0.4921 - acc: 0.5091
Epoch 42/100
55/55 [=======================

...

55/55 [==============================] - 0s 538us/step - loss: 0.4909 - acc: 0.5091
Epoch 97/100
55/55 [==============================] - 0s 545us/step - loss: 0.4909 - acc: 0.5091
Epoch 98/100
55/55 [==============================] - 0s 566us/step - loss: 0.4909 - acc: 0.5091
Epoch 99/100
55/55 [==============================] - 0s 568us/step - loss: 0.4909 - acc: 0.5091
Epoch 100/100
1/1 [==============================] - 2s 2s/step
------ 26 / 56回 -----
Epoch 1/100
55/55 [==============================] - 5s 93ms/step - loss: 0.3589 - acc: 0.4909
Epoch 2/100
55/55 [==============================] - 0s 494us/step - loss: 0.3762 - acc: 0.5455
Epoch 3/100
55/55 [==============================] - 0s 465us/step - loss: 0.3685 - acc: 0.5455
Epoch 4/100
55/55 [==============================] - 0s 401us/step - loss: 0.3545 - acc: 0.5636
Epoch 5/100
55/55 [==============================] - 0s 441us/step - loss: 0.3494 - acc: 0.4909
Epoch 6/100
55/55 [==============================] - 0s 453u

Epoch 80/100
55/55 [==============================] - 0s 557us/step - loss: 0.1926 - acc: 0.7636
Epoch 81/100
55/55 [==============================] - 0s 518us/step - loss: 0.1912 - acc: 0.7636
Epoch 82/100
55/55 [==============================] - 0s 567us/step - loss: 0.1903 - acc: 0.7636
Epoch 83/100
55/55 [==============================] - 0s 548us/step - loss: 0.1888 - acc: 0.7273
Epoch 84/100
55/55 [==============================] - 0s 503us/step - loss: 0.1905 - acc: 0.7636
Epoch 85/100
55/55 [==============================] - 0s 500us/step - loss: 0.1872 - acc: 0.7818
Epoch 86/100
55/55 [==============================] - 0s 520us/step - loss: 0.1893 - acc: 0.7455
Epoch 87/100
55/55 [==============================] - 0s 535us/step - loss: 0.1888 - acc: 0.7636
Epoch 88/100
55/55 [==============================] - 0s 440us/step - loss: 0.1859 - acc: 0.7455
Epoch 89/100
55/55 [==============================] - 0s 507us/step - loss: 0.1833 - acc: 0.7818
Epoch 90/100
55/55 [==========

55/55 [==============================] - 0s 504us/step - loss: 0.2249 - acc: 0.6727
Epoch 64/100
55/55 [==============================] - 0s 469us/step - loss: 0.2232 - acc: 0.7273
Epoch 65/100
55/55 [==============================] - 0s 484us/step - loss: 0.2218 - acc: 0.7273
Epoch 66/100
55/55 [==============================] - 0s 492us/step - loss: 0.2256 - acc: 0.6727
Epoch 67/100
55/55 [==============================] - 0s 426us/step - loss: 0.2206 - acc: 0.6727
Epoch 68/100
55/55 [==============================] - 0s 443us/step - loss: 0.2192 - acc: 0.7273
Epoch 69/100
55/55 [==============================] - 0s 473us/step - loss: 0.2215 - acc: 0.6364
Epoch 70/100
55/55 [==============================] - 0s 442us/step - loss: 0.2211 - acc: 0.6545
Epoch 71/100
55/55 [==============================] - 0s 458us/step - loss: 0.2166 - acc: 0.7455
Epoch 72/100
55/55 [==============================] - 0s 452us/step - loss: 0.2163 - acc: 0.6909
Epoch 73/100
55/55 [=======================

Epoch 47/100
55/55 [==============================] - 0s 516us/step - loss: 0.2201 - acc: 0.6727
Epoch 48/100
55/55 [==============================] - 0s 511us/step - loss: 0.2149 - acc: 0.7273
Epoch 49/100
55/55 [==============================] - 0s 514us/step - loss: 0.2138 - acc: 0.7273
Epoch 50/100
55/55 [==============================] - 0s 511us/step - loss: 0.2135 - acc: 0.7091
Epoch 51/100
55/55 [==============================] - 0s 507us/step - loss: 0.2129 - acc: 0.6727
Epoch 52/100
55/55 [==============================] - 0s 1ms/step - loss: 0.2122 - acc: 0.6727
Epoch 53/100
55/55 [==============================] - 0s 480us/step - loss: 0.2108 - acc: 0.6545
Epoch 54/100
55/55 [==============================] - 0s 490us/step - loss: 0.2092 - acc: 0.7273
Epoch 55/100
55/55 [==============================] - 0s 494us/step - loss: 0.2097 - acc: 0.7273
Epoch 56/100
55/55 [==============================] - 0s 475us/step - loss: 0.2097 - acc: 0.6909
Epoch 57/100
55/55 [============

KeyboardInterrupt: 